In [337]:
import numpy as np
import pandas as pd
import pybaseball
import seaborn as sns
import matplotlib.pyplot as plot
from deepdiff import DeepDiff
from pybaseball import bwar_pitch
from pybaseball import bwar_bat
from pybaseball import cache
from pybaseball.lahman import *
from pybaseball import chadwick_register
from pybaseball import pitching_stats
from pybaseball import batting_stats
from time import sleep

cache.enable()


In [338]:
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 100)

# We're getting the band back together

## All the people plus extra IDs that'll come in handy

In [339]:
people = pd.DataFrame(people())
sleep(1)
people = people.convert_dtypes()
print(people.shape)
print('---')
people.sample(3)

(20543, 24)
---


,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
8314,hinespa01,1855,3,1,USA,VA,<NA>,1935,7,10,USA,MD,Hyattsville,Paul,Hines,Paul Aloysius,173,69,R,R,1872-04-20,1891-07-03,hinep101,hinespa01
17646,stantmi03,1989,11,8,USA,CA,Panorama City,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Giancarlo,Stanton,Giancarlo Cruz-Michael,245,78,R,R,2010-06-08,2022-06-30,stanm004,stantmi03
8242,higgipj01,1993,5,10,USA,CT,Bridgeport,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,P. J.,Higgins,Patrick James,195,70,R,R,2021-05-19,2022-07-01,higgp001,higgipj01


In [340]:
rosetta = pd.DataFrame(chadwick) 
sleep(1)
rosetta = rosetta.convert_dtypes()
print(rosetta.shape)
print('---')
rosetta.sample(3)

(24258, 8)
---


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
18904,Royster,Jerry,121520,roysj001,roystje01,1011270,1973,1988
22558,Vizcaino,Luis,150430,vizcl001,vizcalu01,714,1999,2009
16127,O'Farrell,Bob,119947,ofarb101,o'farbo01,1009740,1915,1935


In [341]:
everyone = pd.merge(rosetta, people, left_on='key_bbref', right_on='playerID', how='left')

In [342]:
print(everyone.shape)
print('---')
everyone.sample(3)

(24258, 32)
---


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
4285,Conover,Theodore,112595,conot101,conovte01,1002539,1889,1889,conovte01,1868,3,10,USA,KY,Lexington,1910,7,27,USA,KY,Paris,Theodore,Conover,Theodore,165,70,R,R,1889-05-26,1889-05-26,conot101,conovte01
20851,Steenstra,Kennie,132640,steek001,steenke01,1012455,1998,1998,steenke01,1970,10,13,USA,MO,Springfield,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Kennie,Steenstra,Kenneth Gregory,220,77,R,R,1998-05-21,1998-06-07,steek001,steenke01
23520,Williams,Pop,124345,willp101,willipo01,1014030,1898,1903,willipo01,1874,5,19,USA,ME,Bowdoinham,1959,8,4,USA,ME,Topsham,Pop,Williams,Walter Merrill,190,71,R,L,1898-09-14,1903-09-15,willp101,willipo01


## Add Fangraphs fielding spaghetti

In [389]:
everyone = everyone.convert_dtypes()

In [390]:
snapshot = everyone

In [391]:
# fielding stats by year 
fielding = pd.DataFrame(fielding()) # this is fine

sleep(1)

TypeError: 'DataFrame' object is not callable

In [392]:
fielding = fielding.convert_dtypes()

In [393]:
fielding.shape

(147080, 18)

In [394]:
snapshot.shape

(24258, 32)

In [395]:
fielding.sample()

,playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
57226,groatdi01,1961,1,PIT,NL,SS,144,143,3767,235,473,32,117,<NA>,<NA>,<NA>,<NA>,<NA>


In [396]:
snapshot = pd.merge(snapshot, fielding, left_on="key_bbref", right_on='playerID', how='left')

In [397]:
print(snapshot.shape)
print('---')
snapshot.sample(30)

(150057, 50)
---


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,playerID_x,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,playerID_y,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
48696,Gilbert,Logan,669302,gilbl002,gilbelo01,22250,2021,2022,gilbelo01,1997,5,5,USA,FL,Winter Park,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Logan,Gilbert,Logan Keith,215,78,R,R,2021-05-13,2022-06-30,gilbl002,gilbelo01,gilbelo01,2021,1,SEA,AL,P,24,24,358,5,9,0,0,<NA>,<NA>,10,0,<NA>
137563,Underwood,Duane,621249,unded001,underdu01,14677,2018,2022,underdu01,1994,7,20,USA,NC,Raleigh,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Duane,Underwood,Duane Lee,225,74,R,R,2018-06-25,2022-06-11,unded001,underdu01,underdu01,2018,1,CHN,NL,P,1,1,12,1,1,0,0,<NA>,<NA>,<NA>,<NA>,<NA>
63650,Howard,Paul,116213,howap101,howarpa01,1006088,1909,1909,howarpa01,1884,5,20,USA,MA,Boston,1968,8,29,USA,FL,Miami,Paul,Howard,Paul Joseph,170,68,R,R,1909-09-16,1909-09-29,howap101,howarpa01,howarpa01,1909,1,BOS,AL,OF,6,4,123,2,1,0,0,<NA>,<NA>,<NA>,<NA>,<NA>
76936,LaValliere,Mike,117543,lavam001,lavalmi01,1007387,1984,1995,lavalmi01,1960,8,18,USA,NC,Charlotte,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Mike,LaValliere,Michael Eugene,180,70,L,R,1984-09-09,1995-09-22,lavam001,lavalmi01,lavalmi01,1993,1,PIT,NL,C,1,1,27,12,0,0,0,1,<NA>,1,0,<NA>
118223,Ruth,Babe,121578,ruthb101,ruthba01,1011327,1914,1935,ruthba01,1895,2,6,USA,MD,Baltimore,1948,8,16,USA,NY,New York,Babe,Ruth,George Herman,215,74,L,L,1914-07-11,1935-05-30,ruthb101,ruthba01,ruthba01,1932,1,NYA,AL,OF,128,<NA>,<NA>,209,10,9,1,<NA>,<NA>,<NA>,<NA>,<NA>
110583,Ramirez,Ramon,430673,ramir003,ramirra02,7986,2006,2014,ramirra02,1981,8,31,D.R.,Santiago,Santiago,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ramon,Ramirez,Ramon Emilio,200,71,R,R,2006-04-14,2014-06-30,ramir003,ramirra02,ramirra02,2010,1,BOS,AL,P,44,0,127,1,5,0,0,<NA>,<NA>,<NA>,<NA>,<NA>
1235,Alcaraz,Luis,110115,alcal101,alcarlu01,1000111,1967,1970,alcarlu01,1941,6,20,P.R.,<NA>,Humacao,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Luis,Alcaraz,Angel Luis,165,69,R,R,1967-09-13,1970-06-07,alcal101,alcarlu01,alcarlu01,1970,1,KCA,AL,2B,31,29,816,65,68,1,12,<NA>,<NA>,<NA>,<NA>,<NA>
132878,Tartabull,Jose,123097,tartj101,tartajo01,1012830,1962,1970,tartajo01,1938,11,27,Cuba,Cienfuegos,Cienfuegos,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Jose,Tartabull,Jose Milages,165,71,L,L,1962-04-10,1970-07-07,tartj101,tartajo01,tartajo01,1967,1,BOS,AL,OF,83,47,1485,90,3,1,1,<NA>,<NA>,<NA>,<NA>,<NA>
43505,Ford,Hod,114303,fordh101,fordho01,1004221,1919,1933,fordho01,1897,7,23,USA,CT,New Haven,1977,1,29,USA,MA,Winchester,Hod,Ford,Horace Hills,165,70,R,R,1919-09-08,1933-05-08,fordh101,fordho01,fordho01,1920,1,BSN,NL,1B,4,<NA>,<NA>,24,1,0,2,<NA>,<NA>,<NA>,<NA>,<NA>
28487,Cox,Dick,112753,cox-d101,coxdi01,1002697,1925,1926,coxdi01,1895,9,30,USA,CA,Pasadena,1966,6,1,USA,CA,Morro Bay,Dick,Cox,Elmer Joseph,158,67,R,R,1925-04-16,1926-09-26,cox-d101,coxdi01,coxdi01,1926,1,BRO,NL,OF,117,<NA>,<NA>,201,12,8,2,<NA>,<NA>,<NA>,<NA>,<NA>


## Add BBRef Pitching Spaghetti

In [399]:
# bbref pitching WAR
bref_pitch = pd.DataFrame(bwar_pitch(return_all=True))
bref_pitch = bref_pitch.convert_dtypes()
sleep(1)

In [400]:
bref_pitch.columns

Index(['name_common', 'age', 'mlb_ID', 'player_ID', 'year_ID', 'team_ID',
       'stint_ID', 'lg_ID', 'G', 'GS', 'IPouts', 'IPouts_start',
       'IPouts_relief', 'RA', 'xRA', 'xRA_sprp_adj', 'xRA_extras_adj',
       'xRA_def_pitcher', 'PPF', 'PPF_custom', 'xRA_final', 'BIP', 'BIP_perc',
       'RS_def_total', 'runs_above_avg', 'runs_above_avg_adj',
       'runs_above_rep', 'RpO_replacement', 'GR_leverage_index_avg', 'WAR',
       'salary', 'teamRpG', 'oppRpG', 'pyth_exponent', 'waa_win_perc', 'WAA',
       'WAA_adj', 'oppRpG_rep', 'pyth_exponent_rep', 'waa_win_perc_rep',
       'WAR_rep', 'ERA_plus', 'ER_lg'],
      dtype='object')

In [401]:
bref_pitch = bref_pitch.rename(columns={
    "mlb_ID": "key_mlabm",
    "player_ID": "key_bbref",
    "year_ID": "Season"
})

In [405]:
a = np.intersect1d(snapshot.columns, bref_pitch.columns)
print(list(a))

['G', 'GS', 'key_bbref']


In [403]:
test = snapshot

In [407]:
test = pd.merge(test, bref_pitch, left_on=['G', 'yearID', 'key_bbref'], right_on=['G', 'Season', 'key_bbref'], how='outer')

In [408]:
test.shape

(155144, 91)

## Add BBRef Batting Spaghetti

In [415]:
# bbref batting WAR
bref_bat = pd.DataFrame(bwar_bat(return_all=True))
bref_bat = bref_bat.convert_dtypes()
sleep(1)

In [417]:
bref_bat = bref_bat.rename(columns={
    "mlb_ID": "key_mlabm",
    "player_ID": "key_bbref",
    "year_ID": "Season"
})

In [418]:
a = np.intersect1d(test.columns, bref_bat.columns)
print(list(a))

['G', 'Season', 'WAA', 'WAR', 'WAR_rep', 'age', 'key_bbref', 'key_mlabm', 'lg_ID', 'name_common', 'oppRpG', 'oppRpG_rep', 'pyth_exponent', 'pyth_exponent_rep', 'runs_above_avg', 'runs_above_rep', 'salary', 'stint_ID', 'teamRpG', 'team_ID', 'waa_win_perc', 'waa_win_perc_rep']


In [371]:
bref_bat.sample()

,name_common,age,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg
10926,Clete Boyer,32,111319,boyercl02,1969,ATL,1,NL,562,144,1198.3,-5.78,-4.63,-1.49,14.8,0,0,0,<NA>,14.8,2.86,0.0,19.2,25.0,5.8,-9.0,17.7,0.76,-1.03,2.09,2.72,2.09,0.93,1.96,<NA>,N,4.00531,4.06809,0.07037,3.93473,1.813,1.809,0.5045,0.493,0.5138,0.4849,95.186294,184.698,191.456


In [372]:
bref_bat = bref_bat.rename(columns={
    "mlb_ID": "key_mlabm",
    "player_ID": "key_bbref",
    "year_ID": "Season"
})

In [373]:
a = np.intersect1d(snapshot.columns, bref_bat.columns)
print(list(a))

['G', 'PA', 'Season', 'WAA', 'WAR', 'WAR_rep', 'age', 'key_bbref', 'key_mlabm', 'lg_ID', 'name_common', 'oppRpG', 'oppRpG_rep', 'pyth_exponent', 'pyth_exponent_rep', 'runs_above_avg', 'runs_above_rep', 'salary', 'stint_ID', 'teamRpG', 'team_ID', 'waa_win_perc', 'waa_win_perc_rep']


In [420]:
test = pd.merge(test, bref_bat, left_on=['G', 'Season', 'WAA', 'WAR', 'WAR_rep', 'age', 'key_bbref', 'key_mlabm', 'lg_ID', 'name_common', 'oppRpG', 'oppRpG_rep', 'pyth_exponent', 'pyth_exponent_rep', 'runs_above_avg', 'runs_above_rep', 'salary', 'stint_ID', 'teamRpG', 'team_ID', 'waa_win_perc', 'waa_win_perc_rep'], right_on=['G', 'Season', 'WAA', 'WAR', 'WAR_rep', 'age', 'key_bbref', 'key_mlabm', 'lg_ID', 'name_common', 'oppRpG', 'oppRpG_rep', 'pyth_exponent', 'pyth_exponent_rep', 'runs_above_avg', 'runs_above_rep', 'salary', 'stint_ID', 'teamRpG', 'team_ID', 'waa_win_perc', 'waa_win_perc_rep'], how='outer')

In [421]:
test.shape

(274863, 118)

In [429]:
print(list(test.columns))

['name_last', 'name_first', 'key_mlbam', 'key_retro', 'key_bbref', 'key_fangraphs', 'mlb_played_first', 'mlb_played_last', 'playerID_x', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry', 'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay', 'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID', 'playerID_y', 'yearID', 'stint', 'teamID', 'lgID', 'POS', 'G', 'GS_x', 'InnOuts', 'PO', 'A', 'E', 'DP', 'PB', 'WP', 'SB', 'CS', 'ZR', 'name_common', 'age', 'key_mlabm', 'Season', 'team_ID', 'stint_ID', 'lg_ID', 'GS_y', 'IPouts', 'IPouts_start', 'IPouts_relief', 'RA', 'xRA', 'xRA_sprp_adj', 'xRA_extras_adj', 'xRA_def_pitcher', 'PPF', 'PPF_custom', 'xRA_final', 'BIP', 'BIP_perc', 'RS_def_total', 'runs_above_avg', 'runs_above_avg_adj', 'runs_above_rep', 'RpO_replacement', 'GR_leverage_index_avg', 'WAR', 'salary', 'teamRpG', 'oppRpG', 'pyth_exponent', 'waa_win_perc', 'WAA', 'W

In [430]:
test.sample(10)

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,playerID_x,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,playerID_y,yearID,stint,teamID,lgID,POS,G,GS_x,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,name_common,age,key_mlabm,Season,team_ID,stint_ID,lg_ID,GS_y,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,WAR,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg,PA,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_avg_off,runs_above_avg_def,WAA_off,WAA_def,WAR_def,WAR_off,pitcher,oppRpPA_rep,waa_win_perc_off,waa_win_perc_def,OPS_plus,TOB_lg,TB_lg
170140,<NA>,<NA>,<NA>,<NA>,buttsto01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Tommy Butts,18,<NA>,1938,BCA,0,NAL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,<NA>,3.1,<NA>,<NA>,0.3,<NA>,5.40829,5.55189,1.979,0.5,0.01,<NA>,5.42751,1.98,0.4888,0.29,<NA>,<NA>,91,<NA>,-4.82,-0.29,0.0,3.6,<NA>,<NA>,<NA>,<NA>,3.6,1.52,0.0,3.11,-3.6,5.1,-0.33,0.46,0.46,-0.04,N,0.10691,0.487,0.5184,61.027043,29.817,31.042
3088,Allen,Johnny,110152,allej102,allenjo02,1000150,1932,1944,allenjo02,1904,9,30,USA,NC,Lenoir,1959,3,29,USA,FL,St. Petersburg,Johnny,Allen,John Thomas,180,72,R,R,1932-04-19,1944-09-26,allej102,allenjo02,allenjo02,1940,1,CLE,AL,P,32,<NA>,<NA>,8,21,0,2,<NA>,<NA>,<NA>,<NA>,<NA>,Johnny Allen,35,110152,1940,CLE,1,AL,17,416,327,89,61,75.718,0.0,<NA>,3.978,97,97.012,69.596,463,0.102,39.0,8.596,7.513,23.136,0.222,1.8253,2.0,11500,5.02788,4.7931,1.918,0.5229,0.7328,-0.058,5.48405,1.955,0.4577,1.3204,123.924528,65.68,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
13704,Boisclair,Bruce,111160,boisb101,boiscbr01,1001131,1974,1979,boiscbr01,1952,12,9,USA,CT,Putnam,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Bruce,Boisclair,Bruce Armand,185,74,L,L,1974-09-11,1979-09-30,boisb101,boiscbr01,boiscbr01,1977,1,NYN,NL,OF,91,65,1871,140,1,6,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6979,Baker,Scott,435044,bakes002,bakersc02,6176,2005,2015,bakersc02,1981,9,19,USA,LA,Shreveport,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Scott,Baker,Timothy Scott,215,76,R,R,2005-05-07,2015-05-02,bakes002,bakersc02,bakersc02,2011,1,MIN,AL,P,23,21,404,11,6,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Scott Baker,29,435044,2011,MIN,1,AL,21,404,395,9,50,69.979,2.344,<NA>,-4.565,99,99.438,76.456,389,0.0821,-55.6,26.456,25.889,40.249,0.199,0.155,4.21,5000000,4.43662,3.31101,1.792,0.6282,2.9486,-0.1221,5.03506,1.898,0.4402,1.3817,129.051064,60.654,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
165207,<NA>,<NA>,<NA>,<NA>,boolere01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Red Booles,28,111212,1909,CLE,1,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<

In [432]:
bat = pd.DataFrame(pybaseball.batting_stats(start_season=1921, end_season=2021))
bat = bat.convert_dtypes()
sleep(1)

In [433]:
bat.sample()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
1119,1014357,1950,Eddie Yost,WAS,23,155,573,731,169,130,26,2,11,114,58,141,<NA>,63,8,<NA>,9,13,6,6,0.295,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.193,0.086,2.24,0.44,0.405,0.845,0.11,0.317,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.403,34.2,125,25.4,-11.0,22.8,1.9,41.0,3.9,<NA>,3.4,125,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-9.1,-1.7,<NA>,23 - 23,23.7,3.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.294,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,106,168,102,120,97,79,109,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>


In [190]:
fpit = pd.DataFrame(pybaseball.pitching_stats(start_season=1921, end_season=2021))
sleep(3)

In [191]:
ftpit = pd.DataFrame(pybaseball.team_pitching(start_season=1921, end_season=2021))
sleep(3)

In [192]:
ftbat = pd.DataFrame(pybaseball.team_batting(start_season=1921, end_season=2021))
sleep(3)

In [193]:
ftfld = pd.DataFrame(pybaseball.team_fielding(start_season=1921, end_season=2021))
sleep(3)

In [333]:

# bref batting WAR
bwar_bat = pd.DataFrame(bwar_bat(return_all=True))
sleep(3)

In [334]:
bwar_pitch.shape

(53666, 43)

In [196]:
bwar_pitch.sort_values('year_ID').head(5)

,name_common,age,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,G,GS,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,WAR,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg
31095,John McMullin,22.0,118836.0,mcmuljo01,1871,TRO,1,NaN,29,29,747,NaN,NaN,362,283.800,0.0,NaN,-10.974,102,NaN,300.669,1243.0,0.9976,-11.0,-61.331,-56.033,-7.505,0.457,1.0,-0.77,NaN,10.36538,12.29755,2.434,0.3975,-2.9725,-0.1468,12.21150,2.431,0.4017,2.3516,77.946405,119.258
45427,Bill Stearns,18.0,122703.0,stearbi01,1871,OLY,1,NaN,2,2,54,NaN,NaN,11,21.162,0.0,NaN,1.439,100,NaN,19.723,67.0,0.0514,28.0,8.723,9.106,12.614,0.457,1.0,0.72,NaN,10.36538,5.81238,2.211,0.7823,0.5646,-0.0106,12.30047,2.434,0.3973,0.1693,172.200000,8.610
29742,Bobby Mathews,19.0,118419.0,mathebo01,1871,KEK,1,NaN,19,19,507,NaN,NaN,243,201.522,0.0,NaN,-24.000,108,NaN,243.564,833.0,1.0000,-24.0,0.564,4.159,37.097,0.457,1.0,1.73,NaN,10.36538,10.14648,2.365,0.5126,0.2394,-0.0996,12.27783,2.433,0.3984,1.5917,88.404124,85.752
44924,Al Spalding,20.0,122558.0,spaldal01,1871,BOS,1,NaN,31,31,772,NaN,NaN,272,289.515,0.0,NaN,2.770,101,NaN,289.612,1216.0,0.9233,3.0,17.612,23.087,73.240,0.457,1.0,3.64,1500.0,10.36538,9.62064,2.348,0.5437,1.3547,-0.1517,12.15019,2.429,0.4047,2.4385,127.121875,122.037
28619,Denny Mack,21.0,118083.0,mackde01,1871,ROK,1,NaN,3,1,39,NaN,NaN,30,15.315,0.0,NaN,-0.361,100,NaN,15.676,66.0,0.0602,-6.0,-14.324,-14.047,-11.514,0.457,1.0,-0.53,NaN,10.36538,15.04771,2.514,0.2815,-0.6555,-0.0077,11.29709,2.403,0.4485,0.1284,125.320000,6.266


In [198]:
appearances = pd.DataFrame(appearances())
sleep(3)

In [201]:
def drop_y(df):
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)


----
^^^^^ Start over from here ^^^^^^

In [215]:
a = np.intersect1d(df.columns, fielding.columns)
print(list(a))

['CS', 'G', 'SB', 'playerID']


In [232]:
snapshot = df
snapshot.shape

(6443, 346)

In [231]:
snapshot = pd.merge(snapshot, fielding, on=['CS', 'G', 'SB', 'playerID'], how='inner')

In [229]:
snapshot.sample(10)

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,name_last,name_first,key_mlbam,key_retro,key_bbref,mlb_played_first,mlb_played_last,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameGiven,weight,height,bats,throws,debut,finalGame,yearID,stint,teamID,lgID,POS,GS,InnOuts,PO,A,E,DP,PB,WP,ZR
104,97.0,2003.0,Chipper Jones,ATL,31.0,153,555.0,656.0,169.0,107.0,33.0,2.0,27.0,103.0,106.0,94.0,13.0,83.0,1.0,6.0,0.0,10.0,2.0,2.0,0.305,180.0,188.0,110.0,16.0,2416.0,1076.0,1340.0,6.0,0.0,0.0,0.143,0.127,1.13,0.402,0.517,0.92,0.213,0.315,0.0096,0.23,0.377,0.393,0.085,0.144,0.033,0.0,0.391,34.9,115.0,31.0,-6.8,20.3,-6.7,39.9,3.9,$16.9,2.9,137.0,4.5,-9.6,14.11,47.06,4.59,1.02,2.65,3.0,3.37,1.06,0.629,90.2,0.083,83.5,NaN,NaN,0.09,76.2,0.179,81.9,0.019,82.9,NaN,NaN,0.013,NaN,31.1,1.4,NaN,-1.1,5.8,-2.1,NaN,2.07,0.72,NaN,-0.53,1.35,-4.46,NaN,0.142,0.739,0.434,0.416,0.873,0.797,0.489,0.541,0.087,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-13.5,-0.6,-0.9,31 - 31,31.9,1.3,2.4,0.44,0.281,0.279,0.115,0.59,0.296,0.311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [186]:
fielding.sample()

,playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
40276,tebbebi01,1937,1,DET,AL,C,48,NaN,NaN,155,25,7.0,1,4.0,NaN,NaN,NaN,NaN


In [184]:
df = df.merge(fielding, on=['CS', 'G', 'SB', 'playerID'], how='left')

In [185]:
df.sample(10)

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,name_last,name_first,key_mlbam,key_retro,key_bbref,mlb_played_first,mlb_played_last,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameGiven,weight,height,bats,throws,debut,finalGame,yearID,stint,teamID,lgID,POS,GS,InnOuts,PO,A,E,DP,PB,WP,ZR
1966,9847.0,2018.0,Andrew McCutchen,- - -,31.0,155,569.0,682.0,145.0,92.0,30.0,3.0,20.0,83.0,65.0,95.0,1.0,145.0,11.0,7.0,0.0,12.0,14.0,9.0,0.255,178.0,154.0,99.0,14.0,2889.0,1176.0,1713.0,14.0,0.0,0.0,0.139,0.213,0.66,0.368,0.424,0.792,0.169,0.304,0.0116,0.23,0.413,0.357,0.091,0.13,0.079,0.0,0.347,18.1,98.0,17.0,-6.8,20.4,-7.1,19.5,2.0,$16.0,4.2,121.0,1.81,-10.74,12.55,14.63,1.68,1.0,0.27,2.0,1.58,0.23,0.543,92.8,0.191,84.3,0.064,88.4,0.094,78.9,0.099,83.7,0.008,85.5,0.001,78.8,0.002,NaN,23.1,-4.4,-0.9,2.6,-0.7,-0.3,-0.2,1.48,-0.8,-0.47,0.98,-0.26,-1.38,-5.41,0.193,0.641,0.398,0.653,0.845,0.795,0.458,0.56,0.082,-5.0,0.33,0.096,0.064,0.009,NaN,0.116,0.185,0.07,0.033,NaN,0.096,NaN,0.001,NaN,93.1,92.7,88.6,84.8,NaN,91.9,84.4,77.5,81.6,NaN,83.9,NaN,78.7,-1.9,-2.4,0.5,-2.8,NaN,-4.0,1.2,0.8,1.4,NaN,0.3,NaN,-0.4,9.2,6.0,5.2,2.9,NaN,4.9,1.2,-4.5,-4.1,NaN,4.1,NaN,-3.0,7.5,7.7,2.4,-0.1,NaN,3.4,-7.3,4.0,-1.7,NaN,0.8,NaN,-0.2,0.78,2.78,1.28,-0.5,NaN,1.03,-1.37,2.01,-1.82,NaN,0.28,NaN,-5.41,0.141,0.623,0.398,0.595,0.832,0.793,0.533,21.8,-13.9,-1.3,-4.2,31 - 31,11.9,1.0,0.5,0.439,0.364,0.197,0.135,0.432,0.434,0.381,0.094,NaN,0.091,0.341,0.075,0.012,0.001,NaN,0.198,0.186,NaN,83.8,NaN,78.7,93.2,88.2,84.4,78.7,NaN,92.3,84.3,NaN,0.5,NaN,1.2,-1.5,0.7,-3.1,0.1,NaN,-3.

In [63]:
# merge magic
stats = pd.merge(fbat, fpit,  how='left', left_on=['IDfg'], right_on=['IDfg'], suffixes=('', '_y'))
sleep(1)
drop_y(stats)
stats.shape

(44707, 319)

In [64]:
# merge magic
stats = pd.merge(stats, ftpit,  how='outer', left_on=['IDfg'], right_on=['IDfg'], suffixes=('', '_y'))
sleep(1)
drop_y(stats)
stats.shape

(421589, 319)

In [ ]:
# merge magic
stats = pd.merge(stats, ftpit,  how='left', left_on=['IDfg'], right_on=['IDfg'], suffixes=('', '_y'))
sleep(1)
drop_y(stats)
stats.shape

(421589, 319)

In [26]:
stats.sample(30)

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
16201,194,1988,Rickey Henderson,NYY,29,140,554,647,169,131,30,2,6,118,50,82,1.0,54,3,6.0,2,6.0,93,13.0,0.305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.127,0.083,1.52,0.394,0.399,0.793,0.094,0.326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.362,24.3,95,20.2,3.0,18.5,-6.3,50.7,5.5,NaN,7.7,127,6.76,-8.75,15.51,48.85,5.08,1.02,1.98,1.0,3.78,2.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.3,13.0,NaN,29 - 29,33.1,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118,152,58,121,102,71,115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
34252,293,1994,Fred McGriff,ATL,30,113,424,478,135,75,25,1,34,81,94,50,8.0,76,1,3.0,0,8.0,7,3.0,0.318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.105,0.159,0.66,0.389,0.623,1.012,0.304,0.319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.423,36.2,97,34.7,6.0,15.0,-6.8,50.2,4.8,NaN,3.8,156,

In [192]:
# merge stats into rosetta, rename as result
result = pd.merge(rosetta, stats,  how='left', left_on=['IDfg'], right_on=['IDfg'], suffixes=('', '_y'))
drop_y(result)

In [193]:
# merge people into result
result = pd.merge(result, people,  how='left', left_on=['key_bbref'], right_on=['playerID'], suffixes=('', '_y'))
drop_y(result)

In [194]:
# merge fielding into result ~90 seconds runtime
result = pd.merge(result, fielding,  how='left', left_on=['key_bbref'], right_on=['playerID'], suffixes=('', '_y'))
drop_y(result)

In [195]:
# merge fielding into result ~4 minutes runtime
result = pd.merge(result, bwar_pitch,  how='left', left_on=['key_bbref'], right_on=['player_ID'], suffixes=('', '_y'))
drop_y(result)

KeyboardInterrupt: 

In [ ]:
result.shape

(11591259, 404)

In [ ]:
result = result.sort_values('WAR', ascending=False)

In [ ]:
result = result.drop_duplicates(subset='key_bbref')

In [ ]:
everyone = result 

In [77]:
everyone = everyone.convert_dtypes()

In [78]:
everyone = everyone.loc[:,~everyone.columns.duplicated()].copy()

In [79]:
everyone = everyone.rename(columns={
    "SF": "sac_flies",
    "SH": "sac_hits",
    "BU": "bunt_attempts",
    "BUH": "bunt_hits",
    "Dol": "pay_me",
    "pLi": "leverage",
    "phLi": "pinch_leverage",
    "WPA/Li": "situational_wins",
    "WPA/Li": "situational_wins",
    "Def": "defensive_RAA",
    "UBR": "ultimate_baserunning"
})

In [80]:
everyone.shape 

(22894, 404)

In [82]:
#THIS WORKS DON'T TOUCH ANYTHING ABOVE HERE OR I WILL SCREAM
print(everyone.columns.tolist())

['name_last', 'name_first', 'key_mlbam', 'key_retro', 'key_bbref', 'IDfg', 'mlb_played_first', 'mlb_played_last', 'Season', 'Name', 'Team', 'Age', 'G', 'AB', 'PA', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'sac_flies', 'sac_hits', 'GDP', 'SB', 'CS', 'AVG', 'GB', 'FB', 'LD', 'IFFB', 'Pitches', 'Balls', 'Strikes', 'IFH', 'bunt_attempts', 'bunt_hits', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'OPS', 'ISO', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%', 'BUH%', 'wOBA', 'wRAA', 'wRC', 'Bat', 'Fld', 'Rep', 'Pos', 'RAR', 'WAR', 'pay_me', 'Spd', 'wRC+', 'WPA', '-WPA', '+WPA', 'RE24', 'REW', 'pLI', 'phLI', 'PH', 'WPA/LI', 'Clutch', 'FB% (Pitch)', 'FBv', 'SL%', 'SLv', 'CT%', 'CTv', 'CB%', 'CBv', 'CH%', 'CHv', 'SF%', 'SFv', 'KN%', 'KNv', 'XX%', 'PO%', 'wFB', 'wSL', 'wCT', 'wCB', 'wCH', 'wSF', 'wKN', 'wFB/C', 'wSL/C', 'wCT/C', 'wCB/C', 'wCH/C', 'wSF/C', 'wKN/C', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwSt

----

# NOW LET'S GET WEIRD
## Don't touch anything above the line.

In [84]:
# Constants
KG_TO_LB = 0.453592
M_TO_IN = 0.0254


In [85]:
### Add BMI Calcs
# BMI Calculations
everyone["KG"] = everyone["weight"] * KG_TO_LB
everyone["meters"] = everyone["height"] * M_TO_IN
everyone["BMI"] = everyone["KG"] / everyone["meters"] ** 2
everyone["ratio"] = everyone["meters"] * everyone["BMI"]

In [171]:
everyone.sample(15)

,name_last,name_first,key_mlbam,key_retro,key_bbref,IDfg,mlb_played_first,mlb_played_last,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,sac_flies,sac_hits,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,bunt_attempts,bunt_hits,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,pay_me,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,defensive_RAA,wSB,ultimate_baserunning,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,yearID,stint,teamID,lgID,POS,GS,InnOuts,PO,A,E,DP,PB,WP,ZR,name_common,age,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg,KG,meters,BMI,ratio
2970846,Fleming,Bill,114227,flemb101,flemibi01,1004144,1940,1946,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [38]:
print(everyone.columns.tolist())

['name_last', 'name_first', 'key_mlbam', 'key_retro', 'key_bbref', 'IDfg', 'mlb_played_first', 'mlb_played_last', 'Season', 'Name', 'Team', 'Age', 'G', 'AB', 'PA', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'sac_flies', 'sac_hits', 'GDP', 'SB', 'CS', 'AVG', 'GB', 'FB', 'LD', 'IFFB', 'Pitches', 'Balls', 'Strikes', 'IFH', 'bunt_attempts', 'bunt_hits', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'OPS', 'ISO', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%', 'BUH%', 'wOBA', 'wRAA', 'wRC', 'Bat', 'Fld', 'Rep', 'Pos', 'RAR', 'WAR', 'pay_me', 'Spd', 'wRC+', 'WPA', '-WPA', '+WPA', 'RE24', 'REW', 'pLI', 'phLI', 'PH', 'WPA/LI', 'Clutch', 'FB% (Pitch)', 'FBv', 'SL%', 'SLv', 'CT%', 'CTv', 'CB%', 'CBv', 'CH%', 'CHv', 'SF%', 'SFv', 'KN%', 'KNv', 'XX%', 'PO%', 'wFB', 'wSL', 'wCT', 'wCB', 'wCH', 'wSF', 'wKN', 'wFB/C', 'wSL/C', 'wCT/C', 'wCB/C', 'wCH/C', 'wSF/C', 'wKN/C', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwSt


# OPEN TRYOUTS ARE HERE


In [155]:
df_save = everyone

In [161]:
df = df_save[
    [
        "playerID",
        "Name",
        "name_last",
        "name_first",
        "POS",
        "BMI",
        "WAR",
        "wRC+",
        "RE24",
        "ERA_plus",
        "IPouts",
        "IPouts_start",
        "IPouts_relief",
        "WAA",
        "WPA",
        "bats",
        "throws",
        "height",
        "weight",
        "Age",
        "Season",
        "yearID",
        "Team",
        "GS",
        "name_last",
        "name_first",
        "birthYear",
        "deathYear",
        "AVG+",
        "BB%+",
        "K%+",
        "OBP+",
        "SLG+",
        "ISO+",
        "BABIP+",
        "AB",
        "PA",
        "H",
        "1B",
        "2B",
        "3B",
        "HR",
        "R",
        "RBI",
        "BB",
        "IBB",
        "SO",
        "HBP",
        "sac_flies",
        "sac_hits",
        "GDP",
        "SB",
        "CS",
        "AVG",
        "BB%",
        "K%",
        "BB/K",
        "OBP",
        "SLG",
        "BIP",
        "BIP_perc",
        "RS_def_total",
        "runs_above_avg",
        "runs_above_avg_adj",
        "runs_above_rep",
        "RpO_replacement",
        "GR_leverage_index_avg",
        "teamRpG",
        "oppRpG",
        "pyth_exponent",
        "waa_win_perc",
        "WAA",
        "WAA_adj",
        "oppRpG_rep",
        "pyth_exponent_rep",
        "waa_win_perc_rep",
        "WAR_rep",
        "ER_lg",
        "OPS",
        "ISO",
        "BABIP",
        "RA",
        "xRA",
        "xRA_sprp_adj",
        "xRA_extras_adj",
        "xRA_def_pitcher",
        "key_mlbam",
        "key_retro",
        "key_bbref",
        "IDfg",
        "salary",
        "pay_me",
    ]
]

In [162]:
everyone.shape

(22894, 408)

In [170]:
everyone.sample(25)

,name_last,name_first,key_mlbam,key_retro,key_bbref,IDfg,mlb_played_first,mlb_played_last,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,sac_flies,sac_hits,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,bunt_attempts,bunt_hits,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,pay_me,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,defensive_RAA,wSB,ultimate_baserunning,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,yearID,stint,teamID,lgID,POS,GS,InnOuts,PO,A,E,DP,PB,WP,ZR,name_common,age,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg,KG,meters,BMI,ratio
4063904,Hamilton,L.,-1,<NA>,hamill01,-1,1924,1924,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [163]:
df.shape

(22894, 92)

In [164]:
df = df.dropna(subset=['BMI']) # Anyone without a BMI doesn't make the first round of cuts.

In [165]:
df.shape

(19465, 92)

In [166]:
pool = df[ # Let's separate the meat from the chaff
    (df.BMI > df.BMI.quantile(0.97))
    & (df.height < df.height.quantile(0.25))]

In [167]:
pool.POS.value_counts() # These are the players who are in the 97th percentile for weight and the 25th percentile for height. 

C     29
P     18
2B    13
OF     9
3B     7
1B     5
SS     2
Name: POS, dtype: Int64

In [168]:
candidates = pool.sort_values("weight", ascending=False)

In [169]:
candidates

,playerID,Name,name_last,name_first,POS,BMI,WAR,wRC+,RE24,ERA_plus,IPouts,IPouts_start,IPouts_relief,WAA,WPA,bats,throws,height,weight,Age,Season,yearID,Team,GS,name_last,name_first,birthYear,deathYear,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,sac_flies,sac_hits,GDP,SB,CS,AVG,BB%,K%,BB/K,OBP,SLG,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ER_lg,OPS,ISO,BABIP,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,key_mlbam,key_retro,key_bbref,IDfg,salary,pay_me
9280604,sandopa01,Pablo Sandoval,Sandoval,Pablo,3B,38.45357,5.2,146,37.44,<NA>,3,0,3,0.0507,4.54,B,R,70,268,22,2009,2017,SFG,25,Sandoval,Pablo,1986,<NA>,124,89,75,114,132,144,116,572,633,189,115,44,5,25,79,90,52,13,83,4,5,0,10,5,5,0.33,0.082,0.131,0.63,0.387,0.556,2,0.0005,27.0,0.472,0.487,0.565,0.207,0.0,4.72014,4.23314,1.868,0.5507,0.0507,-0.0275,4.81473,1.902,0.4906,0.0091,0.634,0.943,0.226,0.35,0,0.467,-0.032,<NA>,0.014,467055,sandp001,sandopa01,5409,555000,$33.1
6728535,moronre01,<NA>,Moronta,Reyes,P,38.02312,<NA>,<NA>,<NA>,166.05,20,0,20,0.1589,<NA>,R,R,70,265,<NA>,<NA>,2017,<NA>,0,Moronta,Reyes,1993,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,15,0.0033,-39.0,1.514,1.544,2.127,0.204,0.3657,4.66123,4.44066,1.877,0.5227,0.1589,-0.0678,4.749,1.894,0.4912,0.0611,3.321,<NA>,<NA>,<NA>,2,3.526,-0.247,<NA>,-0.129,606625,moror001,moronre01,14966,<NA>,<NA>
5382943,kirkal01,<NA>,Kirk,Alejandro,C,37.25171,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,R,R,68,245,<NA>,<NA>,2020,<NA>,6,Kirk,Alejandro,1998,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,672386,kirka001,kirkal01,22581,<NA>,<NA>
7480993,penabr01,<NA>,Pena,Brayan,C,35.441412,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,B,R,69,240,<NA>,<NA>,2005,<NA>,7,Pena,Brayan,1982,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,430910,penab002,penabr01,3231,<NA>,<NA>
710953,belloed01,<NA>,Bellorin,Edwin,C,34.436033,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,R,R,70,240,<NA>,<NA>,2007,<NA>,1,Bellorin,Edwin,1982,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,431143,belle002,belloed01,7242,<NA>,<NA>
5664695,leonsa01,<NA>,Leon,Sandy,C,33.718616,<NA>,<NA>,<NA>,58.92,18,0,18,-0.2214,<NA>,B,R,70,235,<NA>,<NA>,2012,<NA>,9,Leon,Sandy,1989,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,23,0.0057,46.0,-2.237,-2.222,-1.665,0.202,0.0,4.57192,4.94226,1.9,0.4631,-0.2214,0.0941,4.66804,1.885,0.4902,0.0589,2.946,<NA>,<NA>,<NA>,5,3.167,-0.206,0.0,0.262,506702,leons001,leonsa01,5273,1250000,<NA>
7819514,ramirha02,<NA>,Ramirez,Harold,OF,33.288165,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,R,R,70,232,<NA>,<NA>,2019,<NA>,100,Ramirez,Harold,1994,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>

In [ ]:
huskies_P = (
    df_huskies_tryouts_P.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(13, "weight")
)  # 13 heaviest P
huskies_P

In [194]:
pool.groupby()

,height,weight,BMI
count,5289.000000,5289.000000,5289.000000
mean,69.024579,188.852713,27.857111
std,1.290663,13.521389,1.672563
min,55.000000,155.000000,26.544442
25%,68.000000,180.000000,26.581059
50%,69.000000,185.000000,27.319422
75%,70.000000,195.000000,28.409727
max,70.000000,268.000000,38.453570


In [180]:
height_weight.sample(4)

107957    25.423271
77572     23.734004
97324     25.090233
6927      30.430618
Name: BMI, dtype: float64

In [173]:
candidates = df[(df.BMI > df.BMI.quantile(0.75)) & (df.height < df.height.quantile(.25))] # Filter height/weight
candidates

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common_x,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,name_common_y,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg_adj,RpO_replacement,GR_leverage_index_avg,WAA_adj,ERA_plus,ER_lg,Glf,Gcf,Grf,teamID,lgID,POS,GS_y,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
161,abernbi01,1929.0,1.0,30.0,USA,CA,Torrance,2006.0,2.0,19.0,USA,CA,Yucaipa,Bill,Abernathie,William Edward,190.0,70.0,R,R,1952-09-27,1952-09-27,aberb101,abernbi01,86.182480,1.7780,27.261859,48.471586,110019.0,1000018.0,1952.0,1952.0,Bill Abernathie,23.0,1952.0,CLE,1.0,AL,1.0,1.0,NaN,-0.23,0.00,0.00,0.0,NaN,NaN,NaN,NaN,0.0,0.00,0.11,0.00,-0.1,-0.1,-0.1,0.0,-0.01,-0.01,0.00,-0.01,0.00,-0.01,0.00,NaN,Y,4.04466,4.16466,0.07507,4.13399,1.822,1.828,0.4867,0.4867,0.5000,0.4966,-100.000000,0.330,0.368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLE,AL,P,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
244,abreuto01,1984.0,11.0,13.0,D.R.,Puerto Plata,Puerto Plata,NaN,NaN,NaN,NaN,NaN,NaN,Tony,Abreu,Etanislao Toni,200.0,70.0,B,R,2007-05-22,2014-07-28,abret001,abreuto01,90.718400,1.7780,28.696694,51.022722,473234.0,5053.0,2007.0,2014.0,Tony Abreu,22.0,2007.0,LAD,1.0,NL,178.0,59.0,358.0,-5.14,0.68,-0.03,8.0,0.0,0.0,0.0,0.0,8.0,1.50,0.00,5.55,10.6,5.0,-3.0,9.5,0.48,-0.32,0.94,1.00,0.94,0.20,0.52,NaN,N,4.72784,4.77852,0.08882,4.68448,1.900,1.897,0.5086,0.4949,0.5162,0.4906,82.124031,61.143,72.691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,abreuto01,1984.0,11.0,13.0,D.R.,Puerto Plata,Puerto Plata,NaN,NaN,NaN,NaN,NaN,NaN,Tony,Abreu,Etanislao Toni,200.0,70.0,B,R,2007-05-22,2014-07-28,abret001,abreuto01,90.718400,1.7780,28.696694,51.022722,473234.0,5053.0,2007.0,2014.0,Tony Abreu,24.0,2009.0,LAD,1.0,NL,11.0,6.0,19.0,0.15,-0.44,0.00,2.0,0.0,0.0,0.0,0.0,2.0,0.05,0.00,0.34,2.1,1.8,-0.2,2.1,0.19,-0.03,0.22,0.23,0.22,0.01,0.04,400000.0,N,4.46898,4.50898,0.08274,4.45183,1.869,1.868,0.5313,0.4958,0.5364,0.4940,97.498982,3.652,3.301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,abreuto01,1984.0,11.0,13.0,D.R.,Puerto Plata,Puerto Plata,NaN,NaN,NaN,NaN,NaN,NaN,Tony,Abreu,Etanislao Toni,200.0,70.0,B,R,2007-05-22,2014-07-28,abret001,abreuto01,90.718400,1.7780,28.696694,51.022722,473234.0,5053.0,2007.0,2014.0,Tony Abreu,25.0,2010.0,ARI,1.0,NL,201.0,81.0,338.3,-15.23,-0.13,-1.46,-2.0,0.0,0.0,0.0,0.0,-2.0,1.46,0.00,6.26,-11.1,-17.4,-15.4,-0.5,-1.90,-1.68,-0.09,-1.30,-0.09,-1.08,0.60,407000.0,N,4.19157,4.38120,0.08031,4.30386,1.845,1.852,0.4770,0.4796,0.4993,0.4918,46.968202,68.038,81.388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,abreuto01,1984.0,11.0,13.0,D.R.,Puerto Plata,Puerto Plata,NaN,NaN,NaN,NaN,NaN,NaN,Tony,Abreu,Etanislao Toni,200.0,70.0,B,R,2007-05-22,2014-07-28,abret001,abreuto01,90.718400,1.7780,28.696694,51.022722,473234.0,5053.0,2007.0,2014.0,Tony Abreu,27.0,2012.0,KCR,1.0,AL,74.0,22.

birthYear                                                                  \
        count         mean        std     min     25%     50%     75%     max   
POS                                                                             
C         7.0  1881.285714  45.294276  1864.0  1864.0  1864.0  1864.5  1984.0   
SS        8.0  1974.000000   0.000000  1974.0  1974.0  1974.0  1974.0  1974.0   

    birthMonth                                         birthDay             \
         count mean       std  min  25%  50%  75%  max    count       mean   
POS                                                                          
C          7.0  5.0  1.732051  2.0  4.5  6.0  6.0  6.0      7.0  22.285714   
SS         8.0  5.0  0.000000  5.0  5.0  5.0  5.0  5.0      8.0  25.000000   

                                             deathYear                         \
           std   min   25%   50%   75%   max     count         mean       std   
POS                                                                             
C    11.643514   2.0  19.0  29.0  29.0  29.0       6.0  1936.166667  5.307228   
SS    0.000000  25.0  25.0  25.0  25.0  25.0       0.0          NaN       NaN   

                                            deathMonth                      \
        min     25%     50%     75%     max      count      mean       std   
POS                                                                          
C    1934.0  1934.0  1934.0  1934.0  1947.0        6.0  7.666667  0.816497   
SS      NaN     NaN     NaN     NaN     NaN        0.0       NaN       NaN   

                             deathDay                                          \
     min  25%  50%  75%  max    count      mean       std  min  25%  50%  75%   
POS                                                                             
C    6.0  8.0  8.0  8.0  8.0      6.0  9.666667  4.082483  8.0  8.0  8.0  8.0   
SS   NaN  NaN  NaN  NaN  NaN      0.0       NaN       NaN  NaN  NaN  NaN  NaN   

          weight                                                           \
      max  count        mean       std    min    25%    50%    75%    max   
POS                                                                         
C    18.0    7.0  214.714286  0.755929  213.0  215.0  215.0  215.0  215.0   
SS    NaN    8.0  220.000000  0.000000  220.0  220.0  220.0  220.0  220.0   

    height                                                      KG             \
     count       mean      std   min   25%   50%   75%   max count       mean   
POS                                                                             
C      7.0  68.285714  0.48795  68.0  68.0  68.0  68.5  69.0   7.0  97.392682   
SS     8.0  69.000000  0.00000  69.0  69.0  69.0  69.0  69.0   8.0  99.790240   

                                                                 meters  \
          std        min       25%       50%       75%       max  count   
POS                                                                       
C    0.342883  96.615096  97.52228  97.52228  97.52228  97.52228    7.0   
SS   0.000000  99.790240  99.79024  99.79024  99.79024  99.79024    8.0   

                                                                  BMI  \
         mean       std     min     25%     50%     75%     max count   
POS                                                                     
C    1.734457  0.012394  1.7272  1.7272  1.7272  1.7399  1.7526   7.0   
SS   1.752600  0.000000  1.7526  1.7526  1.7526  1.7526  1.7526   8.0   

                                                                      \
          mean       std        min        25%        50%        75%   
POS                                                                    
C    32.379319  0.537861  31.454253  32.219937  32.690276  32.690276   
SS   32.487961  0.000000  32.487961  32.487961  32.487961  32.487961   

               ratio                                                        \
           max count       mean

In [16]:
df["BMI"] = round(df.BMI, 2)  # clean up our BMI decimals
df = df.dropna(subset=["BMI"])  # Drop anyone without a weight.
df_huskies_tryouts = df[df["weight"] >= 225]

df_huskies_tryouts = df_huskies_tryouts.sort_values("weight", ascending=False)

In [17]:
# weight of our heaviest starting lineup
df_huskies_tryouts.groupby("POS").weight.max().sum()

1892.0

In [18]:
df_huskies_tryouts

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common_x,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,name_common_y,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg_adj,RpO_replacement,GR_leverage_index_avg,WAA_adj,ERA_plus,ER_lg,Glf,Gcf,Grf,teamID,lgID,POS,GS_y,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
109928,youngwa01,1980.0,2.0,18.0,USA,MS,Hattiesburg,2015.0,9.0,19.0,USA,MS,Purvis,Walter,Young,Walter Ernest,320.0,77.0,L,R,2005-09-06,2005-10-02,younw001,youngwa01,145.14944,1.9558,37.95,74.214869,425488.0,2112.0,2005.0,2005.0,Walter Young,25.0,2005.0,BAL,1.0,AL,37.0,14.0,81.0,0.47,-0.12,0.19,0.0,0.0,0.0,0.0,0.0,0.0,-0.81,0.00,1.39,1.1,-0.3,-0.3,-0.8,-0.03,-0.03,-0.07,0.10,-0.07,0.10,0.13,NaN,N,4.69790,4.71718,0.08197,4.61808,1.895,1.890,0.4981,0.4981,0.4942,0.4900,115.029797,12.192,13.972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25063,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,30.0,2014.0,CIN,1.0,NL,0.0,34.0,34.7,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,-0.03,0.00,-0.03,0.00,0.00,NaN,Y,3.99035,3.99035,0.07088,3.99035,1.808,1.808,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25064,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,31.0,2015.0,CIN,1.0,NL,0.0,58.0,60.3,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,-0.01,0.00,-0.01,0.00,0.00,510000.0,Y,4.22222,4.22222,0.07613,4.22222,1.837,1.837,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25065,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,32.0,2016.0,CIN,1.0,NL,1.0,45.0,43.0,-0.28,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.14,0.00,-0.1,-0.1,-0.1,0.0,-0.01,-0.01,-0.02,-0.01,-0.02,-0.01,0.00,525000.0,Y,4.49699,4.50010,0.08241,4.49938,1.870,1.870,0.4997,0.4997,0.5000,0.4999,-100.000000,0.332,0.427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIN,NL,P,0.0,129.0,1.0,3.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN
25066,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,33.0,2017.0,TBR,1.0,AL,0.0,2.0,30.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0

In [ ]:
# Probably easiest if we make ourselves a dataframe for every position, since we'll need to fill up a 26-man roster, which is parsed into several discrete roles that each require a particular number of players. 

# These role counts aren't codified, but teams have nearly always carried the same distribution of player-roles.

df_huskies_tryouts_P = df_huskies_tryouts[df_huskies_tryouts['POS'] == "P"]
df_huskies_tryouts_C = df_huskies_tryouts[df_huskies_tryouts['POS'] == "C"]
df_huskies_tryouts_1B = df_huskies_tryouts[df_huskies_tryouts['POS'] == "1B"]
df_huskies_tryouts_2B = df_huskies_tryouts[df_huskies_tryouts['POS'] == "2B"]
df_huskies_tryouts_3B = df_huskies_tryouts[df_huskies_tryouts['POS'] == "3B"]
df_huskies_tryouts_SS = df_huskies_tryouts[df_huskies_tryouts['POS'] == "SS"]
df_huskies_tryouts_OF = df_huskies_tryouts[df_huskies_tryouts['POS'] == "OF"]


In [ ]:
df_huskies_tryouts_P

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common_x,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,name_common_y,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg_adj,RpO_replacement,GR_leverage_index_avg,WAA_adj,ERA_plus,ER_lg,Glf,Gcf,Grf,teamID,lgID,POS,GS_y,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
25065,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,32.0,2016.0,CIN,1.0,NL,1.0,45.0,43.0,-0.28,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.14,0.0,-0.1,-0.1,-0.1,0.0,-0.01,-0.01,-0.02,-0.01,-0.02,-0.01,0.0,525000.0,Y,4.49699,4.50010,0.08241,4.49938,1.870,1.870,0.4997,0.4997,0.5000,0.4999,-100.000000,0.332,0.427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIN,NL,P,0.0,129.0,1.0,3.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN
11968,brownju01,1907.0,4.0,30.0,USA,RI,Greene,1966.0,10.0,2.0,USA,NY,Freeport,Jumbo,Brown,Walter George,295.0,76.0,R,R,1925-08-26,1941-08-27,browj110,brownju01,133.80964,1.9304,35.91,69.317053,111597.0,1001533.0,1925.0,1941.0,Jumbo Brown,31.0,1938.0,NYG,1.0,NL,18.0,43.0,NaN,-1.13,-0.52,-0.24,0.0,NaN,NaN,NaN,NaN,0.0,0.00,2.07,0.0,0.2,0.2,0.2,0.0,0.01,0.02,-0.03,0.01,-0.03,0.02,0.0,NaN,Y,4.42789,4.42371,0.07970,4.41010,1.862,1.861,0.5004,0.5004,0.5000,0.4986,45.477489,6.116,6.277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NY1,NL,P,NaN,NaN,5.0,11.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN
11967,brownju01,1907.0,4.0,30.0,USA,RI,Greene,1966.0,10.0,2.0,USA,NY,Freeport,Jumbo,Brown,Walter George,295.0,76.0,R,R,1925-08-26,1941-08-27,browj110,brownju01,133.80964,1.9304,35.91,69.317053,111597.0,1001533.0,1925.0,1941.0,Jumbo Brown,30.0,1937.0,NYG,2.0,NL,0.0,4.0,NaN,0.00,0.00,0.00,0.0,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,NaN,Y,4.55686,4.55686,0.08283,4.55686,1.877,1.877,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NY1,NL,P,NaN,NaN,0.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
11969,brownju01,1907.0,4.0,30.0,USA,RI,Greene,1966.0,10.0,2.0,USA,NY,Freeport,Jumbo,Brown,Walter George,295.0,76.0,R,R,1925-08-26,1941-08-27,browj110,brownju01,133.80964,1.9304,35.91,69.317053,111597.0,1001533.0,1925.0,1941.0,Jumbo Brown,32.0,1939.0,NYG,1.0,NL,12.0,31.0,NaN,0.28,-0.46,0.17,0.0,NaN,NaN,NaN,NaN,0.0,0.00,1.32,0.0,1.3,1.3,1.3,0.0,0.14,0.14,-0.02,0.14,-0.02,0.14,0.0,7500.0,Y,4.51438,4.47213,0.08040,4.45954,1.870,1.866,0.5044,0.5044,0.5000,0.4987,110.357093,4.168,4.425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NY1,NL,P,NaN,NaN,0.0,12.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
11970,brownju01,1907.0,4.0,30.0,USA,RI,Greene,1966.0,10.0,2.0,USA,NY,Freeport,Jumbo,Brown,Walter George,295.0,76.0,R,R,1925-08-26,1941-08-27,browj110,brownju01,133.80964,1.9304,35.91,69.317053,111597.0,1001533.0,1925.0,1941.0,Jumbo Brown,33.0,1940.0,NYG,1.0,NL,10.0,41.0,NaN,-1.89,0.00,0.00,0.0,NaN,NaN,NaN,NaN,0.0,0.00,1.10,0.0,-0.8,-0.8,-0.8,0.0,-0.08,-0.08,-0.01,

We'll need:
- 5 starting pitchers
- 7 relievers
- 2 catchers
- 4 starting infielders
- 2 utility infielders
- 5 outfielders


In [ ]:
huskies_P = (
    df_huskies_tryouts_P.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(13, "weight")
)  # 13 heaviest P

In [ ]:
huskies_C = (
    df_huskies_tryouts_C.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(2, "weight")
)  # 2 heaviest catchers

In [ ]:
huskies_1B = (
    df_huskies_tryouts_1B.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(1, "weight")
)  # heaviest 1B

In [ ]:
huskies_2B = (
    df_huskies_tryouts_2B.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(2, "weight")
)  # 2 heaviest 2B

In [ ]:
huskies_3B = (
    df_huskies_tryouts_3B.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(1, "weight")
)  # 1 heaviest 3B

In [ ]:
huskies_SS = (
    df_huskies_tryouts_SS.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(2, "weight")
)  # 2 heaviest SS

In [ ]:
huskies_OF = (
    df_huskies_tryouts_OF.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(5, "weight")
)  # 5 heaviest OF

In [ ]:
huskies = pd.concat(
    [huskies_P, huskies_C, huskies_1B, huskies_2B, huskies_3B, huskies_SS, huskies_OF],
    ignore_index=True,
)

In [ ]:
# BEHOLD. THE WONKAVILLE HUSKIES.
huskies.POS.value_counts().sum()

26

In [ ]:
df = huskies.dropna(axis=1)

In [ ]:
print(list(df.columns))

['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry', 'birthCity', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID', 'KG', 'meters', 'BMI', 'ratio', 'mlbID', 'fangraphsID', 'mlb_played_first', 'mlb_played_last', 'name_common_x', 'age', 'year_ID', 'team_ID', 'stint', 'lg_ID', 'PA', 'G', 'runs_bat', 'runs_br', 'runs_dp', 'runs_field', 'runs_defense', 'runs_position', 'runs_position_p', 'runs_replacement', 'runs_above_rep', 'runs_above_avg', 'runs_above_avg_off', 'runs_above_avg_def', 'WAA', 'WAA_off', 'WAA_def', 'WAR', 'WAR_def', 'WAR_off', 'WAR_rep', 'pitcher', 'teamRpG', 'oppRpG', 'oppRpPA_rep', 'oppRpG_rep', 'pyth_exponent', 'pyth_exponent_rep', 'waa_win_perc', 'waa_win_perc_off', 'waa_win_perc_def', 'waa_win_perc_rep', 'TOB_lg', 'TB_lg', 'teamID', 'lgID', 'POS', 'PO', 'A', 'E', 'DP']


In [ ]:
df.runs_position_p.value_counts()

0.00    18
0.14     2
2.57     1
5.86     1
0.27     1
0.15     1
0.16     1
0.29     1
Name: runs_position_p, dtype: int64

In [ ]:
# check to make sure everyone's best season is in here
# make sure we've got a rotation and relievers
# clean up columns
huskies.weight.describe()


count     26.000000
mean     271.423077
std       18.465477
min      230.000000
25%      260.000000
50%      275.000000
75%      281.500000
max      315.000000
Name: weight, dtype: float64

In [ ]:
roster_of = df_huskies_tryouts_OF.groupby("playerID", as_index=False)["WAR"].max()